Data quality in a region

Run this file in the python environment

__________________________________________________

Import the csv datafile here.

In the file explorer right click the dataset and copy path, then paste between the "..."

Any backslashes should be replaced by double backslash.

Make sure that the csv file is seperated by commas.

In [2]:
import pandas as pd

ais = pd.read_csv("Dataset.csv",sep=",") # Paste the datafile here
ais.head()

C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_6316\11124247.py:3: DtypeWarning: Columns (4,9) have mixed types. Specify dtype option on import or set low_memory=False.
  ais = pd.read_csv("Dataset.csv",sep=",") # Paste the datafile here


,Timestamp,Packet Type,Channel,Message Type,MMSI,Navigation Status,Repeat Indicator,IMO,ROT,SOG,...,ETA,A,B,C,D,Prop message 1,Prop message 2,Prop message 3,Prop message 4,Prop message 5
0,2023-10-28 07:17:51.000,BSVDM,A,1.0,353246000,Underway using engine,0.0,0.0,0.0,12.5,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,2023-10-28 07:17:51.278,BSVDO,A,4.0,2579991,NaN,0.0,0.0,NaN,NaN,...,2023-10-28 07:12:19,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,2023-10-28 07:17:52.488,BSADS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,STX564855481005,71220.0,V,0.0,I
3,2023-10-28 07:17:52.657,BSVDM,A,1.0,431697000,Underway using engine,0.0,0.0,0.0,11.3,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,2023-10-28 07:17:53.519,PSTXI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,INFO,2.0,0,0.0,1


The columns needed for this analysis are: "Error", "MMSI", "Position Accuracy", "Latitude", "Longitude"

Subset the dataset for a specific region

In [3]:
ais_subset = ais[ais["Region"]=='QH69']

Check how many error messages the dataset has.

In [ ]:
ais_subset[ais_subset["Error"].notna()]["Error"].value_counts()

# The output will be the number of error messages

Count the values in the ”Accuracy Position” column per unique vessel

In [5]:
cont_table = pd.crosstab(ais_subset['MMSI'], ais_subset['Position Accuracy'])
cont_table

Position Accuracy,<10m,>10m
MMSI,,
205776000.0,0,694
209034000.0,0,3451
209271000.0,0,91
209317000.0,0,400
209623000.0,0,1090
...,...,...
636093017,0,2355
636093025,0,2689
636093050,0,2020


Export the table to csv file if needed

In [6]:
output_filename = 'Contiguency_table_position_accuracy.csv'
cont_table.to_csv(output_filename, index=False) 
# This file will be saved in the same directory as this script.

Check if there are any message that were reported to be on land ( This accuracy is between 100 meter and 1 kilometer.The Global Land Mask library is quite reliable for general applications. It may not be perfect for applications requiring very fine spatial accuracy near coastlines or in areas with complex geographic features.)

Install the library:

In [10]:
pip install global_land_mask

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: global_land_mask in c:\programs files\python\python310\lib\site-packages (1.0.0)



You should consider upgrading via the 'c:\Programs Files\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [15]:
from global_land_mask import globe

# Apply the globe.is_land function row-wise to create the 'Is Land' column
# The column will have boolean values
ais_subset['Is Land'] = ais_subset.apply(lambda row: globe.is_land(row['Latitude'], row['Longitude']), axis=1)

# Check for the results
ais_subset["Is Land"].head()

C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_6316\2481843197.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ais_subset['Is Land'] = ais_subset.apply(lambda row: globe.is_land(row['Latitude'], row['Longitude']), axis=1)


0    False
1    False
3    False
7    False
8    False
Name: Is Land, dtype: bool

Count the values in the ”Is Land” column.

In [16]:
island_table = ais_subset['Is Land'].value_counts().reset_index()
island_table.columns = ['Is Land', 'Count']

island_table

,Is Land,Count
0,False,4701770
1,True,29758
